Making The Fermi-Hubbard Hamiltonian from Sparse Paulis
==========================================================

This method failed because I did not include the fermionic properties of the creation/annihilation operators and could not figure it out. See [this](../hamiltonian_from_scratch-fermions.ipynb) too see corrected version.

In [1]:
from qiskit import *
from qiskit.quantum_info.operators import SparsePauliOp
from qiskit.quantum_info.operators import Pauli
from qiskit.quantum_info import Operator

import numpy as np
from functools import *
import scipy.linalg as LA

In [2]:
I = SparsePauliOp(Pauli("I"))
# creation = Operator(np.array([[0,0], [1,0]]))
creation = SparsePauliOp.from_list([("X", 0.5),("Y", -0.5j)])
annihilation = SparsePauliOp.from_list([("X", 0.5),("Y", 0.5j)])
number = SparsePauliOp.from_list([("I", 0.5),("Z", -0.5)])

number.to_matrix()

array([[0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])

In [3]:

verticies = [0,1,2,3,4,5]
spins = [0,1]
edges = [(0,1),(0,2),(0,3),(0,4),
         (1,0),(2,0),(3,0),(4,0),
         (5,1),(5,2),(5,3),(5,4),
         (1,5),(2,5),(3,5),(4,5),
         (2,1),(3,2),(4,3),(1,4),
         (1,2),(2,3),(3,4),(4,1)]

# verticies = [0,1]
# spins = [0,1]
# edges = [(0,1), (1,0),]

state_labels = [(vertex, spin) for spin in spins for vertex in verticies]

myTensor = lambda a, b: (a)^(b)

def swap_electron_helper(vertex:int, spin:int, edge:int, s:int) -> SparsePauliOp:
    """The if-else statement for which operator to tensor into the mix. 
    If the spin is s, and the vertex is in edge, return creation or annihilation 
    depending on which vertex it is."""
    (i, j) = edge
    if spin == s:
        if vertex == i:
            return creation
        elif vertex == j:
            return annihilation
    return I

def swap_electron(edge:tuple[int, int], s:int) -> SparsePauliOp:
    """Swaps the electron in spin state s in vertex i and vertex j."""
    (i, j) = edge
    multiplier = 1 if i<j else -1
    swap1 = reduce(myTensor, [swap_electron_helper(vertex, spin, edge, s) 
                              for (vertex, spin) in state_labels])
    return multiplier*swap1

def number_up_down_helper(vertex:int, i:int) -> SparsePauliOp:
    """The if-else statement for the number operator. 
    Vertex matches i, return a number operator."""
    if i == vertex:
        return number
    else:
        return I

def number_up_down(i) -> SparsePauliOp:
    return reduce(myTensor, 
           [number_up_down_helper(vertex, i) for (vertex, _) in state_labels])

total_swap_operator = reduce(lambda a, b: (a) + (b), 
                             [swap_electron(edge, s) for edge in edges for s in spins])
total_number_operator = reduce(lambda a, b: (a) + (b), 
                               [number_up_down(i) for i in verticies])

t = 1
u = 1
hamiltonian = -t*total_swap_operator + u*total_number_operator

In [4]:
print("Dims in hamiltonian:", hamiltonian.dim)
print("hamiltonian hermitian?:", Operator(hamiltonian) == Operator(hamiltonian).adjoint())

Dims in hamiltonian: (4096, 4096)
hamiltonian hermitian?: False


In [56]:
H_matrix = hamiltonian.to_matrix()
eigs = LA.eigh(H_matrix)


In [57]:
eigenvals, eigenvectors = eigs

In [ ]:
[e for e in enumerate(eigenvals)]

[(0, np.float64(-13.129911807875267)),
 (1, np.float64(-12.80531814198162)),
 (2, np.float64(-12.805318141981617)),
 (3, np.float64(-12.313980658158806)),
 (4, np.float64(-11.805318141981617)),
 (5, np.float64(-11.805318141981617)),
 (6, np.float64(-11.648716584155641)),
 (7, np.float64(-11.648716584155638)),
 (8, np.float64(-10.817536297596737)),
 (9, np.float64(-10.817536297596734)),
 (10, np.float64(-10.31398065815881)),
 (11, np.float64(-10.16121260001673)),
 (12, np.float64(-10.16121260001673)),
 (13, np.float64(-9.829477285951908)),
 (14, np.float64(-9.829477285951901)),
 (15, np.float64(-8.9354579136589)),
 (16, np.float64(-8.9354579136589)),
 (17, np.float64(-8.9354579136589)),
 (18, np.float64(-8.829477285951908)),
 (19, np.float64(-8.829477285951905)),
 (20, np.float64(-8.817536297596734)),
 (21, np.float64(-8.817536297596734)),
 (22, np.float64(-8.592801756361974)),
 (23, np.float64(-8.592801756361974)),
 (24, np.float64(-8.592801756361974)),
 (25, np.float64(-8.592801756361

In [101]:
[(f"{float(x):.8f}, {i:012b}") for i,x in enumerate(eigenvectors[:, 0]) if x > 1e-2]


/tmp/ipykernel_20377/3913122665.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  [(f"{float(x):.8f}, {i:012b}") for i,x in enumerate(eigenvectors[:, 0]) if x > 1e-2]


['0.03306383, 000111000111',
 '0.04444305, 000111001011',
 '0.03664636, 000111001101',
 '0.04444305, 000111001110',
 '0.03664636, 000111010011',
 '0.04444305, 000111010101',
 '0.04444305, 000111010110',
 '0.04073598, 000111011001',
 '0.04972796, 000111011010',
 '0.04972796, 000111011100',
 '0.04444305, 000111100011',
 '0.04444305, 000111100101',
 '0.03664636, 000111100110',
 '0.04972796, 000111101001',
 '0.04972796, 000111101010',
 '0.04073598, 000111101100',
 '0.04972796, 000111110001',
 '0.04073598, 000111110010',
 '0.04972796, 000111110100',
 '0.04544855, 000111111000',
 '0.04444305, 001011000111',
 '0.04907679, 001011001011',
 '0.04444305, 001011001101',
 '0.05450749, 001011001110',
 '0.04444305, 001011010011',
 '0.05908010, 001011010101',
 '0.05990652, 001011010110',
 '0.04444305, 001011011001',
 '0.05450749, 001011011010',
 '0.05990652, 001011011100',
 '0.05450749, 001011100011',
 '0.05990652, 001011100101',
 '0.04972796, 001011100110',
 '0.05450749, 001011101001',
 '0.05511490, 

In [41]:
state_labels

[(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 0),
 (0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1)]